<a href="https://colab.research.google.com/github/HugoVenegas23/P2_RandomWalks/blob/main/P2_RandomWalksII_HEGV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#P2 Random Walks II
**Nombre**: Hugo Esteba González Venegas

**e-mail**: hugo.gonzalez0181@alumnos.udg.mx